##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pruning no exemplo do Keras

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/model_optimization/guide/pruning/pruning_with_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/model_optimization/guide/pruning/pruning_with_keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/model_optimization/guide/pruning/pruning_with_keras.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Este é um exemplo completo para o *pruning de peso* baseado em magnitude.

### Outras páginas

Para uma introdução sobre o que é o pruning e para determinar se você deve usá-lo (incluindo a compatibilidade dele), veja a [página de visão geral](https://www.tensorflow.org/model_optimization/guide/pruning).

Para encontrar rapidamente as APIs necessárias para seu caso de uso (além de realizar todo o pruning de um modelo com uma esparsidade de 80%), veja o [guia completo](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md).

### Resumo

Neste tutorial, você:

1. Treinará um modelo `tf.keras` para o dataset MNIST do zero.
2. Ajustará o modelo ao aplicar a API de pruning e verá a exatidão.
3. Criará modelos do TF e do TFLite 3x menores com o pruning.
4. Criará um modelo do TFLite 10x menor ao combinar o pruning e a quantização pós-treinamento.
5. Verá a persistência da exatidão do TF para o TFLite.

## Configuração

In [ ]:
! pip install -q tensorflow-model-optimization

In [ ]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

## Treine um modelo para o MNIST sem o pruning

In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

Avalie a exatidão do teste de referência e salve o modelo para usar depois.

In [ ]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

## Ajuste o modelo pré-treinado com o pruning


### Defina o modelo

Você aplicará o pruning a um modelo inteiro e verá isso no resumo do modelo.

Nesse exemplo, o modelo começa com 50% da esparsidade (50% de zeros nos pesos) e termina com 80% da esparsidade.

No [guia completo](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md), você pode ver como fazer o pruning de algumas camadas para melhorar a exatidão do modelo.

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

### Treine e compare o modelo com a referência

Ajuste com o pruning para duas épocas.

`tfmot.sparsity.keras.UpdatePruningStep` é necessário durante o treinamento, e `tfmot.sparsity.keras.PruningSummaries` fornece logs para monitorar o progresso e depurar.

In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]
  
model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Nesse exemplo, há uma perda mínima na exatidão do teste após o pruning, em comparação com a referência.

In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Os logs mostram a progressão da esparsidade em cada camada.

In [ ]:
#docs_infra: no_execute
%tensorboard --logdir={logdir}

Para usuários fora do Colab, é possível ver os [resultados de uma execução anterior](https://tensorboard.dev/experiment/sRQnrycaTMWQOaswXzClYA/#scalars&_smoothingWeight=0) desse bloco de código no [TensorBoard.dev](https://tensorboard.dev/).

## Crie modelos 3x menores com o pruning

O <code>tfmot.sparsity.keras.strip_pruning</code> e a aplicação de um algoritmo de compressão padrão (por exemplo, por gzip) são necessários para ver os benefícios de compressão do pruning.

- `strip_pruning` é necessário, já que remove toda tf.Variable que só é necessária para o pruning durante o treinamento e aumentaria o tamanho do modelo durante a inferência.
- A aplicação de um algoritmo de compressão padrão é necessária, já que as matrizes de pesos serializadas são do mesmo tamanho que antes do pruning. No entanto, o pruning transforma a maioria dos pesos em zeros, uma redundância adicional que os algoritmos podem utilizar para comprimir mais o modelo.

Primeiro, crie um modelo comprimível para o TensorFlow.

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Em seguida, crie um modelo comprimível para o TFLite.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

Defina uma função helper para comprimir os modelos por gzip e medir o tamanho compactado.

In [ ]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

Compare e veja se os modelos estão 3x menores com o pruning.

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

## Crie um modelo 10x menor ao combinar o pruning e a quantização

Você pode aplicar a quantização pós-treinamento ao modelo podado para benefícios adicionais.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

## Veja a persistência da exatidão do TF para o TFLite

Defina uma função helper para avaliar o modelo do TFLite com o dataset de teste.

In [ ]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

Avalie o modelo após o pruning e a quantização e veja se a exatidão do TensorFlow persiste no back-end do TFLite.

In [ ]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

## Conclusão

Neste tutorial, você viu como criar modelos esparsos para ambos o TensorFlow e o TFLite com a API do Kit de ferramentas para otimização de modelos do TensorFlow. Depois, você combinou o pruning com a quantização pós-treinamento para benefícios adicionais.

Você criou um modelo 10x menor para o MNIST, com uma diferença mínima na exatidão.

Recomendamos que você teste essa nova capacidade que pode ser especialmente importante para a implantação em ambientes com recursos limitados.
